In [2]:
import pandas as pd
import numpy as np
import re
import gc  # Garbage Collector zur Speicherverwaltung


def drop_records_brand_equal_model(df):
    """
    Entfernt alle Zeilen, in denen brand und model nach Normalisierung identisch sind.
    """
    def normalize(text):
        if pd.isna(text):
            return ""
        return re.sub(r"[^a-z0-9]", "", text.lower())

    df = df.copy()
    df["brand_norm"]  = df["brand"].apply(normalize)
    df["model_norm"]  = df["model"].apply(normalize)

    # Zeilen behalten, bei denen die Normalisierungen unterschiedlich sind
    df = df[df["brand_norm"] != df["model_norm"]].reset_index(drop=True)

    df.drop(columns=["brand_norm", "model_norm"], inplace=True)
    return df
#--------------------------------------   

def preprocessing_pipeline(path = '../data.csv'):
        
    # Daten laden
    df = pd.read_csv(path)
        
    # entferne Duplikate  
    df = df.drop_duplicates(subset= ['brand', 'model', 'color', 'registration_date', 'year',
       'price_in_euro', 'power_kw', 'power_ps', 'transmission_type',
       'fuel_type', 'fuel_consumption_l_100km', 'fuel_consumption_g_km',
       'mileage_in_km', 'offer_description']) 

    # Droppe zweite Index Spalte
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)

        # alle fuel types werden behalten, aber nochmal Kontrolle dass keine verrutschten Zeilen drin. Deshalb gültige fuel types definiert
    df = df.loc[df['fuel_type'].isin(['Diesel', 'Petrol','Hybrid', 'Diesel Hybrid', 'Electric', 'LPG', 'CNG', 'Ethanol', 'Hydrogen', 'Other'])] 
    df = df.reset_index(drop=True) # Index wird neu nummeriert, da vorher Zeilen rausgelöscht wurden. 
        # drop=True verhindert dass die Indexzahl alleine als neue  Zeile gewertet wird. 
    
            
    # Zeilen mit falschen Jahreszahlen werden herausgenommen
    yearsToFilter = list(df['year'].unique()[:29])
    filt = [val in yearsToFilter for val in df['year']]
    df = df[filt]

    def Electrics_Reichweite(df): # schreibt Recihweite aus g/km als l/100 für alle E Autos
    # Filtere die Zeilen, bei denen "fuel_type" = "Electric" und "fuel_consumption_g_km" das Wort "Reichweite" enthält
        e_mit_reichweite = (df["fuel_type"] == "Electric") & (df["fuel_consumption_g_km"].astype(str).str.contains("Reichweite", na=False))
        # Cutte in der Spalte "fuel_consumption_g_km" beim ersten Leerzeichen
        df.loc[e_mit_reichweite, "fuel_consumption_g_km"] = df.loc[e_mit_reichweite, "fuel_consumption_g_km"].astype(str).str.split().str[0]
        # Kopiere Werte von "fuel_consumption_g_km" in die Spalte "fuel_consumption_l_100km"
        df.loc[e_mit_reichweite, "fuel_consumption_l_100km"] = df.loc[e_mit_reichweite, "fuel_consumption_g_km"]
        
        return df
        
    df = Electrics_Reichweite(df)
        
    """Problem: diese Funktion muss überarbeitet werden. 
        Her würden wir aktuell in der Zeile if pd.isna(value) or 'l/100 km' not in str(value) alle E Autos rausschmeißen      
        Stattdessen umschreiben als: Für alle Zeilen, wo Wort "l/100km" enthalten ist, dort wie gewohnt beim ersten Leerzeichen kappen und Komma durch Punkt ersetzen
        wenn "l/100km" nicht enthalten ist, aber fuel-Type "Electric" ist, dann einfach so lassen
        wenn "l/100km" nicht enthalten ist und  fuel-Type nicht "Electric" ist, dann auf nan setzen
        """
    
    return df

In [3]:
df = pd.read_csv('../data.csv')

yearsToFilter = list(df['year'].unique()[:29])
filt = [val in yearsToFilter for val in df['year']]
df = df[filt]

In [4]:
#df= preprocessing_pipeline()

### Distribution

In [5]:
df['fuel_type'].value_counts()

fuel_type
Petrol           143280
Diesel            86421
Hybrid            12607
Electric           5967
LPG                1255
CNG                 508
Diesel Hybrid       476
Other               178
Unknown              96
Hydrogen             82
Ethanol              10
Name: count, dtype: int64

### Hybrid

In [6]:
# Filter auf fuel_type = 'Hybrid'
hybrid_df = df[df['fuel_type'] == 'Hybrid']

# Enthält genau die Zeichenfolge 'l/100 km'
mask_l_per_100km = hybrid_df['fuel_consumption_l_100km'].astype(str).str.contains(r'l/100 km', na=False)
count_l_per_100km = mask_l_per_100km.sum()

# Enthält genau die Zeichenfolge 'km (Ort)'
mask_km_ort = hybrid_df['fuel_consumption_l_100km'].astype(str).str.contains(r'km \(Ort\)', na=False)
count_km_ort = mask_km_ort.sum()

# Gesamtanzahl Hybrid-Zeilen
total_hybrid = len(hybrid_df)

# Prozentanteile berechnen
share_l = count_l_per_100km / total_hybrid * 100
share_km = count_km_ort / total_hybrid * 100

# Ausgabe
print(f"Hybrid mit 'l/100 km': {count_l_per_100km} ({share_l:.2f}%)")
print(f"Hybrid mit 'km (Ort)': {count_km_ort} ({share_km:.2f}%)")

Hybrid mit 'l/100 km': 10212 (81.00%)
Hybrid mit 'km (Ort)': 137 (1.09%)


In [7]:
def clean_fuel_consumption(value): 
        if pd.isna(value) or 'l/100 km' not in str(value):
            return np.nan
        try:
            return float(value.split(' ')[0].replace(',', '.'))
        except:
            return np.nan

In [8]:
# Nur bei bestimmten fuel_types anwenden
mask = df['fuel_type'].isin(['Petrol', 'Diesel', 'Hybrid'])
df.loc[mask, 'fuel_consumption_l_100km'] = df.loc[mask, 'fuel_consumption_l_100km'].apply(clean_fuel_consumption)

In [9]:
df[df['fuel_type'] == 'Hybrid']['fuel_consumption_l_100km'].value_counts()

fuel_consumption_l_100km
1.4     458
4.0     446
3.8     406
3.3     402
4.9     387
       ... 
14.9      1
10.4      1
9.5       1
53.0      1
22.6      1
Name: count, Length: 118, dtype: int64

In [10]:
df[df['fuel_type'] == 'Hybrid'].sort_values(by = 'fuel_consumption_l_100km', ascending= False)

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
83857,83857,hyundai,Hyundai SANTA FE,silver,02/2023,2023,50900,169,230,Automatic,Hybrid,53.0,126 g/km,17.0,Prime Hybrid T-GDI EU6d Facelift HEV 1.6 T-GDi...
250736,250736,volvo,Volvo XC90,grey,04/2022,2022,67950,335,455,Automatic,Hybrid,22.6,34 g/km,38500.0,T8 AWD Recharge Inscription *Head-up*Standhzg*
42917,42917,bmw,BMW 220,grey,04/2023,2023,40990,125,170,Automatic,Hybrid,19.0,144 g/km,1711.0,i Active Tourer Luxury Line Head-Up DAB LED
245955,245955,volkswagen,Volkswagen T7 Multivan,red,01/2023,2023,71281,160,218,Automatic,Hybrid,18.9,42 g/km,8000.0,Energetic eHybrid
40582,40582,bmw,BMW 330,black,07/2020,2020,42880,135,184,Automatic,Hybrid,17.0,38 g/km,23573.0,"e M SPORT HUD,HARMAN/K,KOMFORTZUGANG,LASER"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250939,250939,volvo,Volvo XC90,white,06/2022,2022,77990,228,310,Automatic,Hybrid,NaN,- (g/km),24582.0,T8 long range Recharge R-Design AWD
251032,251032,volvo,Volvo XC60,black,03/2023,2023,73950,228,310,Automatic,Hybrid,NaN,75 km Reichweite,4200.0,Recharge T8 AWD Plug-in Hybrid Ultimate Bright...
251045,251045,volvo,Volvo XC60,black,03/2023,2023,74970,335,455,Automatic,Hybrid,NaN,77 km Reichweite,8000.0,Recharge T8 Plus Bright Standheizung/LED
251074,251074,volvo,Volvo XC40,white,04/2023,2023,57990,192,261,Automatic,Hybrid,NaN,43 km Reichweite,1229.0,Plus Bright T5 Recharge Intellisafe*Surround+P...


### Electric

In [11]:
df[df['fuel_type'] == 'Electric']

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
16552,16552,audi,Audi e-tron,beige,09/2019,2019,51888,300,408,Automatic,Electric,NaN,359 km Reichweite,84800.0,55 Advanced Quattro S-Line Interieur
16559,16559,audi,Audi e-tron,beige,07/2019,2019,53990,300,408,Automatic,Electric,NaN,359 km Reichweite,51000.0,55 quattro advanced Pano B&O AHK Matrix
16561,16561,audi,Audi e-tron,beige,11/2019,2019,54870,300,408,Automatic,Electric,NaN,0 g/km,82814.0,+ADVANCED+55+PANO+LUFT+HU+MATRIX+
16571,16571,audi,Audi e-tron,beige,12/2019,2019,61989,300,408,Automatic,Electric,NaN,0 g/km,55990.0,advanced 55 QU*B&O*MATRIX*PANO*LED*NAV
16579,16579,audi,Audi e-tron,blue,02/2019,2019,32930,300,408,Automatic,Electric,NaN,359 km Reichweite,84300.0,55 qu. S line AHK LED V-Cockp. RüKa Pano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251033,251033,volvo,Volvo C40,black,05/2023,2023,52890,170,231,Automatic,Electric,NaN,400 km Reichweite,8.0,Recharge Single Motor Ultimate AHK PANO
251037,251037,volvo,Volvo XC40,black,04/2023,2023,49900,170,231,Automatic,Electric,NaN,0 g/km,14900.0,Recharge Single Motor Plus
251048,251048,volvo,Volvo C40,black,01/2023,2023,51990,170,231,Automatic,Electric,NaN,0 g/km,2106.0,Recharge Single Motor Plus AHK Sitzhzg.
251056,251056,volvo,Volvo C40,black,05/2023,2023,60520,170,231,Automatic,Electric,NaN,400 km Reichweite,3000.0,Ultimate 2WD Recharge Pure Electric


In [12]:
df[df['fuel_type'] == 'Electric']['fuel_consumption_l_100km'].value_counts()

fuel_consumption_l_100km
0 kWh/100 km       101
389 km (Ort)        12
592 km (Ort)        12
305 km (Ort)        11
155 km (Ort)        10
                  ... 
17,8 kWh/100 km      1
540 km (Ort)         1
257 km (Ort)         1
13 kWh/100 km        1
541 km (Ort)         1
Name: count, Length: 175, dtype: int64

In [13]:
# Filter auf fuel_type = 'Electric'
electric_df = df[df['fuel_type'] == 'Electric']

# Enthält genau die Zeichenfolge 'kWh/100 km'
count_kwh_per_100km = electric_df['fuel_consumption_l_100km'].astype(str).str.contains(r'kWh/100 km', na=False).sum()

# Enthält genau die Zeichenfolge 'km (Ort)'
count_km_ort = electric_df['fuel_consumption_l_100km'].astype(str).str.contains(r'km \(Ort\)', na=False).sum()

# Gesamtanzahl der Electric-Zeilen
total_electric = len(electric_df)

# Prozentanteile berechnen
share_kwh = count_kwh_per_100km / total_electric * 100
share_km_ort = count_km_ort / total_electric * 100

# Ausgabe
print(f"Electric mit 'kWh/100 km': {count_kwh_per_100km} ({share_kwh:.2f}%)")
print(f"Electric mit 'km (Ort)': {count_km_ort} ({share_km_ort:.2f}%)")

Electric mit 'kWh/100 km': 125 (2.09%)
Electric mit 'km (Ort)': 343 (5.75%)


In [14]:
# Filter auf Electric-Fahrzeuge
electric_df = df[df['fuel_type'] == 'Electric']

# Maske für beide enthaltenen Fälle
mask_kwh = electric_df['fuel_consumption_l_100km'].astype(str).str.contains(r'kWh/100 km', na=False)
mask_km_ort = electric_df['fuel_consumption_l_100km'].astype(str).str.contains(r'km \(Ort\)', na=False)

# Kombinierte Maske für alles, was NICHT eines der beiden Muster enthält
mask_other = ~(mask_kwh | mask_km_ort)

# Gefilterter DataFrame
electric_other = electric_df[mask_other]

# Ausgabe
electric_other['fuel_consumption_l_100km'].value_counts()

fuel_consumption_l_100km
- (l/100 km)    3
Name: count, dtype: int64

In [15]:
df[df['fuel_type'] == 'Electric']['fuel_consumption_g_km'].value_counts()

fuel_consumption_g_km
0 g/km               2930
200 km Reichweite     177
230 km Reichweite     120
400 km Reichweite      70
340 km Reichweite      69
                     ... 
710 km Reichweite       1
426 km Reichweite       1
151 km Reichweite       1
262 km Reichweite       1
548 km Reichweite       1
Name: count, Length: 383, dtype: int64

In [16]:
# Filter auf fuel_type = 'Electric'
electric_df = df[df['fuel_type'] == 'Electric']

# Enthält genau die Zeichenfolge 'kWh/100 km'
count_kwh_per_100km = electric_df['fuel_consumption_g_km'].astype(str).str.contains(r'km Reichweite', na=False).sum()

# Enthält genau die Zeichenfolge 'km (Ort)'
'''count_km_ort = electric_df['fuel_consumption_l_100km'].astype(str).str.contains(r'km \(Ort\)', na=False).sum()'''

# Gesamtanzahl der Electric-Zeilen
total_electric = len(electric_df)

# Prozentanteile berechnen
share_kwh = count_kwh_per_100km / total_electric * 100
share_km_ort = count_km_ort / total_electric * 100

# Ausgabe
print(f"Electric mit 'kWh/100 km': {count_kwh_per_100km} ({share_kwh:.2f}%)")
print(f"Electric mit 'km (Ort)': {count_km_ort} ({share_km_ort:.2f}%)")

Electric mit 'kWh/100 km': 3009 (50.43%)
Electric mit 'km (Ort)': 343 (5.75%)


In [17]:
# Filter auf Electric-Fahrzeuge
electric_df = df[df['fuel_type'] == 'Electric']

# Maske für beide enthaltenen Fälle
mask_kwh = electric_df['fuel_consumption_g_km'].astype(str).str.contains(r'km Reichweite', na=False)
#mask_km_ort = electric_df['fuel_consumption_l_100km'].astype(str).str.contains(r'km \(Ort\)', na=False)

# Kombinierte Maske für alles, was NICHT eines der beiden Muster enthält
mask_other = ~(mask_kwh)

# Gefilterter DataFrame
electric_other = electric_df[mask_other]

# Ausgabe
electric_other['fuel_consumption_g_km'].value_counts()

fuel_consumption_g_km
0 g/km          2930
-/-               16
122 g/km           2
13 g/km            1
460 km (Ort)       1
389 km (Ort)       1
152 g/km           1
60 g/km            1
85 km (Ort)        1
90 km (Ort)        1
235 km (Ort)       1
100 km (Ort)       1
230 km (Ort)       1
Name: count, dtype: int64

In [ ]:
e_mit_reichweite = (df['fuel_type'] == 'Electric') & (df['fuel_consumption_g_km'].astype(str).str.contains(r'km Reichweite', na=False))
df.loc[e_mit_reichweite, 'electric_range'] = df.loc[e_mit_reichweite, 'fuel_consumption_g_km'].astype(str).str.split().str[0]

### LPG

In [19]:
df[df['fuel_type'] == 'LPG']

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description,electric_range
1722,1722,audi,Audi S8,silver,08/1997,1997,21899,250,340,Manual,LPG,"13,9 l/100 km",- (g/km),220000.0,4.2 EL.GSD~LEDER~NAVI~XENON~TEMPOMAT~SHZ,NaN
1872,1872,audi,Audi A3,silver,05/2000,2000,3600,132,179,Manual,LPG,"7,8 l/100 km",- (g/km),268300.0,"1.8 Turbo 180 PS, ViALLE LPG, Glas-SSD, AHK, Temp",NaN
1875,1875,audi,Audi A3,silver,05/2000,2000,3600,132,179,Manual,LPG,"7,8 l/100 km",- (g/km),268300.0,"1.8 Turbo 180 PS, ViALLE LPG, Glas-SSD, AHK, Temp",NaN
1892,1892,audi,Audi A3,silver,05/2000,2000,3600,132,179,Manual,LPG,"7,8 l/100 km",- (g/km),268300.0,"1.8 Turbo 180 PS, ViALLE LPG, Glas-SSD, AHK, Temp",NaN
1903,1903,audi,Audi S6,blue,01/2001,2001,8000,250,340,Automatic,LPG,"14,2 l/100 km",341 g/km,335000.0,Avant 4.2 quattro,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240560,240560,volkswagen,Volkswagen Golf,yellow,10/2020,2020,20970,66,90,Manual,LPG,"4,5 l/100 km",102 g/km,19000.0,VIII 1.0l TSI * LED. Digital Cockpit. PDC vo+h...,NaN
245179,245179,volkswagen,Volkswagen T-Cross,orange,03/2022,2022,24870,81,110,Automatic,LPG,5 l/100 km,114 g/km,20000.0,Active DSG AUTOGAS,NaN
246929,246929,volvo,Volvo V40,blue,12/2000,2000,2490,120,163,Automatic,LPG,10 l/100 km,- (g/km),295000.0,2.0t LPG,NaN
247024,247024,volvo,Volvo S60,black,07/2010,2010,5500,102,139,Manual,LPG,"5,8 l/100 km",- (g/km),232000.0,T3,NaN


In [20]:
# Filter auf fuel_type = 'Hybrid'
hybrid_df = df[df['fuel_type'] == 'LPG']

# Enthält genau die Zeichenfolge 'l/100 km'
mask_l_per_100km = hybrid_df['fuel_consumption_l_100km'].astype(str).str.contains(r'l/100 km', na=False)
count_l_per_100km = mask_l_per_100km.sum()

# Enthält genau die Zeichenfolge 'km (Ort)'
mask_km_ort = hybrid_df['fuel_consumption_l_100km'].astype(str).str.contains(r'km \(Ort\)', na=False)
count_km_ort = mask_km_ort.sum()

# Gesamtanzahl Hybrid-Zeilen
total_hybrid = len(hybrid_df)

# Prozentanteile berechnen
share_l = count_l_per_100km / total_hybrid * 100
share_km = count_km_ort / total_hybrid * 100

# Ausgabe
print(f"Hybrid mit 'l/100 km': {count_l_per_100km} ({share_l:.2f}%)")
print(f"Hybrid mit 'km (Ort)': {count_km_ort} ({share_km:.2f}%)")

Hybrid mit 'l/100 km': 1060 (84.46%)
Hybrid mit 'km (Ort)': 0 (0.00%)


In [21]:
# Filter auf Electric-Fahrzeuge
electric_df = df[df['fuel_type'] == 'LPG']

# Maske für beide enthaltenen Fälle
mask_kwh = electric_df['fuel_consumption_l_100km'].astype(str).str.contains(r'l/100 km', na=False)
mask_km_ort = electric_df['fuel_consumption_l_100km'].astype(str).str.contains(r'km \(Ort\)', na=False)

# Kombinierte Maske für alles, was NICHT eines der beiden Muster enthält
mask_other = ~(mask_kwh | mask_km_ort)

# Gefilterter DataFrame
electric_other = electric_df[mask_other]

# Ausgabe
electric_other['fuel_consumption_l_100km'].value_counts()

fuel_consumption_l_100km
14,9 kg/100 km    4
4,9 kg/100 km     2
6,3 kg/100 km     1
8,4 kg/100 km     1
8,3 kg/100 km     1
14,7 kg/100 km    1
Name: count, dtype: int64

In [22]:
# Nur bei bestimmten fuel_types anwenden
mask = df['fuel_type'].isin(['LPG'])
df.loc[mask, 'fuel_consumption_l_100km'] = df.loc[mask, 'fuel_consumption_l_100km'].apply(clean_fuel_consumption)

In [23]:
df[df['fuel_type'] == 'LPG'].sort_values(by = 'fuel_consumption_l_100km', ascending= False)

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description,electric_range
51420,51420,dodge,Dodge Durango,black,11/2018,2018,39900,268,364,Automatic,LPG,18.0,387 g/km,46350.0,R/T 5.7 V8 4x4 Blacktop Package + LPG,NaN
87145,87145,jeep,Jeep Grand Cherokee,white,05/2022,2022,91999,268,364,Automatic,LPG,17.7,386 g/km,18000.0,"L 5,7l Summit,ACC,Pano,LPG",NaN
51860,51860,dodge,Dodge Challenger,black,12/2022,2022,103500,535,727,Automatic,LPG,16.5,388 g/km,10.0,"Hellcat 6.2l,Widebody,ACC,LPG",NaN
51833,51833,dodge,Dodge Challenger,red,12/2022,2022,105000,535,727,Automatic,LPG,16.5,388 g/km,10.0,"Hellcat 6.2l,Widebody,ACC,LPG",NaN
51834,51834,dodge,Dodge Challenger,red,12/2022,2022,103500,535,727,Automatic,LPG,16.5,388 g/km,10.0,"Hellcat 6.2l,Widebody,ACC,LPG",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183842,183842,seat,SEAT Tarraco,grey,11/2020,2020,33940,140,190,Automatic,LPG,NaN,166 g/km,40650.0,Xcellence Pano Leder Kameras Led Ahk 20,NaN
214005,214005,volkswagen,Volkswagen Beetle,blue,06/1999,1999,3999,85,116,Manual,LPG,NaN,- (g/km),283000.0,ABT Ausstattung,NaN
219152,219152,volkswagen,Volkswagen Golf Plus,black,06/2008,2008,3499,59,80,Manual,LPG,NaN,0 g/km,166825.0,Edition1.4 Benzin+ LPG-Gas Klimatronic,NaN
224315,224315,volkswagen,Volkswagen Caddy,grey,06/2012,2012,8600,75,102,Manual,LPG,NaN,- (g/km),224000.0,1.6 TDI Maxi 2 Hand Navi Leder AHK,NaN


### CNG

In [24]:
df[df['fuel_type'] == 'CNG']

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description,electric_range
9864,9864,audi,Audi A3,blue,01/2015,2015,20930,81,110,Automatic,CNG,5 kg/100 km,115 g/km,98674.0,1.4 TFSI g-tron S-tronic S-Line GRA,NaN
10844,10844,audi,Audi A3,silver,07/2015,2015,17200,110,150,Automatic,CNG,"4,6 kg/100 km",- (g/km),80000.0,1.4 TFSI cylinder on demand ultra Limousine At...,NaN
11169,11169,audi,Audi A3,white,03/2015,2015,13990,81,110,Automatic,CNG,5 kg/100 km,115 g/km,133000.0,Ambiente Sportback Erdgas CNG,NaN
11825,11825,audi,Audi A3,grey,10/2016,2016,16950,81,110,Manual,CNG,"3,6 kg/100 km",98 g/km,82880.0,1.4TFSI g-tron 2x S line/Sportback/VOLL/CNG,NaN
12034,12034,audi,Audi A3,black,12/2016,2016,15950,81,110,Automatic,CNG,"5,4 kg/100 km",128 g/km,78500.0,Sportback g-tron*AHK*Erdgas*Automatik,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240568,240568,volkswagen,Volkswagen up!,yellow,12/2020,2020,13490,50,68,Manual,CNG,"5,8 kg/100 km",105 g/km,22700.0,"eco up! 1.0 EcoFuel, Benzin / CNG Temp BT SHz",NaN
241359,241359,volkswagen,Volkswagen Golf Variant,silver,06/2020,2020,20990,96,131,Automatic,CNG,"4,8 kg/100 km",111 g/km,35700.0,"VII 1,5 TGI BENZIN/CNG DSG LED StHei",NaN
246961,246961,volvo,Volvo V70,black,06/2006,2006,4250,103,140,Manual,CNG,9 kg/100 km,- (g/km),326000.0,V70 Bi-Fuel Momentum,NaN
247037,247037,volvo,Volvo V70,black,12/2010,2010,8888,170,231,Automatic,CNG,"9,7 l/100 km",232 g/km,271000.0,2.5T Aut. Momentum,NaN
